### Домашняя работа к уроку 1
### Студент: Абрамов А.В.

In [17]:
import numpy as np
import pandas as pd

#### 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$

In [3]:
# функция для вычисления Recall@k
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list[:k])
    recall_k = flags.sum()/len(bought_list)
    
    return recall_k

In [66]:
# пробные данные
rec = [5, 8, 12, 97, 4, 19, 34, 65, 26, 74]
bought = [12, 60, 88, 5]
price_rec = [20, 30, 10, 15, 40, 5, 70, 100, 60, 40]
price_bought = [10, 95, 60, 20]

r = np.array(rec)
b = np.array(bought)
pr = np.array(price_rec)
pb = np.array(price_bought)

In [10]:
recall_at_k(r, b, k=2)

0.25

In [14]:
# функция для вычисления Money Recall@k
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(bought_list, recommended_list[:k])
    total_bought = flags@price_bought
    money_recall_k = total_bought/prices_bought.sum()
    
    return money_recall_k

In [16]:
money_recall_at_k(r, b, pr, pb, k=2)

0.10810810810810811

#### 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  ReciprocalRank(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [62]:
# функция для вычисления MRR@k
def reciprocal_rank_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    for i in range(k):
        if recommended_list[i] in bought_list:
            rank_k = 1/(i+1)
            break
    
    return rank_k

In [67]:
reciprocal_rank_at_k(r, b, k=5)

1.0

#### 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [59]:
# функция для вычисления nDCG@k
def ndcg_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    log_list = np.array([1/(np.log2(i+2)) for i in range(k)])
    flags = np.isin(recommended_list[:k], bought_list)
    
    ideal = []
    for i in range(k):
        if i<len(bought_list):
            ideal.append(1)
        else:
            ideal.append(0)
    ideal = np.array(ideal)
    
    idcg_k = sum(log_list*ideal)/k
    dcg_k = (log_list@flags)/k
    ndcg_k = dcg_k/idcg_k
    
    return ndcg_k

In [60]:
ndcg_at_k(r, b, k=5)

0.5855700749881525